# Set up Account APIs
Running this script will create the APIs that we will be monitoring for accounts and transactions. They require user authentication. 

The API setups are defined with tags that allow us to easily provide OBIE metric reporting.

## Requirements
1. Create an APImetrics Project
1. Get an API key with EDITOR permissions for the project, enter it below
1. Get the maTLS transport certificate and key in PEM format, enter paths below
1. Run this workbook to create all APIs
1. Create an environment variable "financial_id" - TBD - add this here
1. Create a Worfklow for the Account ID API calls - TDB - add this here


In [ ]:
# Enter intended project's APImetrics API key here:
API_KEY = ""

# Enter the bank's full path, including https and without trailing /
BANK_URL = 'https://example.com/open-banking/3.1/aisp'

SSL_KEY_PATH = "" #"./local/TPP_OB_Transport.key"
SSL_CERT_PATH = "" #"./local/TTP_OB_Transport.pem"

OAUTH_METHOD = "private_key_jwt"
# if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
# # CLIENT_ID = '...'
# # CLIENT_SECRET = '...'

# If you've already created the Auth, put the ID here
AUTH_ID = ""
# If you've already created the Token, put the ID here
TOKEN_ID = ""

# The OBIE version ID we're working against
VERSION = "3.1"

from urllib.parse import urlparse

url_info = urlparse(BANK_URL)
DOMAIN, _, _ = url_info.netloc.partition(':') # removes the port 

In [ ]:
# Helper functions
import requests

class APImetricsAPI:

    BASE_URL = "https://client.apimetrics.io/api/2"
    GET_HEADERS = {"Accept": "application/json"}
    POST_HEADERS = {"Content-Type": "application/json", **GET_HEADERS}

    def __init__(self, api_key):
        self.api_key = api_key

    def headers(self, is_post=False):
        other_headers = self.POST_HEADERS if is_post else self.GET_HEADERS
        return {"Authorization": "Bearer {}".format(self.api_key), **other_headers}

    def get(self, path):
        resp = requests.get(
            "{}/{path}".format(self.BASE_URL, path=path),
            headers=self.headers(),
        )
        try:
            resp.raise_for_status()
        except Exception as ex:
            print(resp.content)
            raise ex
        return resp.json()

    def post(self, path, setup):
        resp = requests.post(
            "{}/{path}".format(self.BASE_URL, path=path),
            json=setup,
            headers=self.headers(True),
        )
        try:
            resp.raise_for_status()
        except Exception as ex:
            print(resp.content)
            raise ex
        return resp.json()

    def get_auths(self):
        return self.get("auth/")

    def get_calls(self):
        return self.get("calls/")

    def get_schedules(self):
        return self.get("schedules/")

    def get_tokens(self):
        return self.get("tokens/")

    def get_token(self, id_str):
        return self.get("tokens/{}".format(id_str))

    def get_tokens_for_auth(self, auth_id):
        return self.get("tokens/auth/{}/".format(auth_id))

    def get_workflows(self):
        return self.get("workflows")

    def create_auth(self, setup):
        return self.post("auth/", setup)

    def create_call(self, setup):
        return self.post("calls/", setup)

    def create_token(self, setup):
        return self.post("tokens/", setup)

    def create_workflow(self, setup):
        return self.post("workflows/", setup)

    def update_auth(self, id_str, setup):
        return self.post("auth/{}/".format(id_str), setup)

    def update_call(self, id_str, setup):
        return self.post("calls/{}/".format(id_str), setup)

    def update_token(self, id_str, setup):
        return self.post("tokens/{}/".format(id_str), setup)

    def update_workflow(self, id_str, setup):
        return self.post("workflows/{}/".format(id_str), setup)

    def get_env_variable(self, env, key):
        return self.get("environment/{env}/{key}".format(env=env, key=key))

    def set_env_variable(self, env, key, val):
        return self.post(
            "environment/{env}/{key}".format(env=env, key=key), {"value": val}
        )

    def run_call(self, id_str, config):
        return self.post("calls/{}/run".format(id_str), config)
    
# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

# All the account APIs share a similar setup, this is a mapping of
# our tag to the OBIE tag, the path and the friendly name
SETUP_MAP = {
    "bulk:accounts": (4, "/accounts", "Accounts: Bulk"),
    "bulk:balances": (7, "/balances", "Balances: Bulk"),
    "bulk:transactions": (9, "/transactions", "Transactions: Bulk"),
    "bulk:beneficiaries": (11, "/beneficiaries", "Beneficiaries: Bulk"),
    "bulk:directdebits": (13, "/direct-debits", "Direct-Debits: Bulk"),
    "bulk:standingorders": (15, "/standing-orders", "Standing-Orders: Bulk"),
    "bulk:products": (17, "/products", "Products: Bulk"),
    "bulk:offers": (19, "/offers", "Offers: Bulk"),
    "bulk:party": (21, "/party", "Party: Bulk"),
    "bulk:scheduledpayments": (23, "/scheduled-payments", "Scheduled-Payments: Bulk"),
    "bulk:statements": (28, "/statements", "Statements: Bulk"),
    "account:account": (5, "/accounts/__ACCOUNT_ID__", "Accounts: Get for AccountId"),
    "account:balance": (6, "/accounts/__ACCOUNT_ID__/balances", "Balances: Get for AccountId"),
    "account:transactions": (8, "/accounts/__ACCOUNT_ID__/transactions", "Transactions: Get for AccountId"),
    "account:beneficiaries": (10, "/accounts/__ACCOUNT_ID__/beneficiaries", "Beneficiaries: Get for AccountId"),
    "account:directdebits": (12, "/accounts/__ACCOUNT_ID__/direct-debits", "Direct-Debits: Get for AccountId"),
    "account:standingorders": (14, "/accounts/__ACCOUNT_ID__/standing-orders", "Standing-Orders: Get for AccountId"),
    "account:product": (16, "/accounts/__ACCOUNT_ID__/product", "Product: Get for AccountId"),
    "account:offers": (18, "/accounts/__ACCOUNT_ID__/offers", "Offers: Get for AccountId"),
    "account:party": (20, "/accounts/__ACCOUNT_ID__/party", "Party: Get for AccountId"),
    "account:scheduledpayments": (22, "/accounts/__ACCOUNT_ID__/scheduled-payments", "Scheduled-Payments: Get for AccountId"),
    "account:statements": (25, "/accounts/__ACCOUNT_ID__/statements/__STATEMENT_ID__", "Statements: Get for StatementId"),
    "account:statement": (24, "/accounts/__ACCOUNT_ID__/statements", "Statements: Get for AccountId"),
    "account:statement_file": (26, "/accounts/__ACCOUNT_ID__/statements/__STATEMENT_ID__/file", "Statements: File for StatementId"),
    "account:statement_transactions": (27, "/accounts/__ACCOUNT_ID__/statements/__STATEMENT_ID__/transactions", "Statements: Transactions for StatementId"),
}

# Helper function - the APImetrics API setup object
def get_call(ob_id, path, name, tag):
    setup = {
        "meta": {
            "description": "Get a list of accounts",
            "tags": [
                "api_type:read",
                "sector:financial",
                "ob_id:{}".format(ob_id),
                "ob_v:{}".format(VERSION),
                tag,
            ],
            "name": "v{}: {}".format(VERSION, name),
            "workspace": "global",
        },
        "request": {
            "body": None,
            "parameters": [],
            "url": "{}{}".format(BANK_URL, path),
            "auth_id": AUTH_ID,
            "headers": [
                {"key": "Accept", "value": "application/json"},
                {"key": "x-fapi-financial-id", "value": "{{financial_id}}"},
            ],
            "token_id": TOKEN_ID,
            "method": "GET",
        },
    }
    return setup

def get_call_setup(call_tag):
    ob_id, path, name = SETUP_MAP[call_tag]
    return get_call(ob_id, path, name, f"bank:{VERSION}:{call_tag}")

## Create Auth Setting and Token for Bank

In [ ]:
# First create Auth Setting
if not AUTH_ID:

    ssl_key = None
    ssl_cert = None

    if SSL_KEY_PATH:
        with open(SSL_KEY_PATH) as stream:
            ssl_key = stream.read()

    if SSL_CERT_PATH:
        with open(SSL_CERT_PATH) as stream:
            ssl_cert = stream.read()
    
    setup = {
        "access": {
            "keys": False,
            "org_keys": False,
            "org_settings": True,
            "settings": False,
        },
        "keys": {},
        "meta": {
            "domain": DOMAIN,
            "documentation": {"keys": "", "docs": "", "apps": "", "provider": ""},
            "name": "Transport MATLS",
            "tags": ['auth:bank_matls'],
            "description": "Mutual Authenticated TLS for calls to bank APIs",
        },
        "settings": {
            "auth_type": "MANUAL",
            "ssl_key": ssl_key,
            "ssl_cert": ssl_cert,
        },
    }
    if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
        setup["keys"]["client_id"] = CLIENT_ID
        setup["keys"]["client_secret"] = CLIENT_SECRET
    
    auth = CLIENT.create_auth(setup)
    AUTH_ID = auth['id']
    print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")
    
# Second Create Token
if not TOKEN_ID:
    setup = {
        'meta': {
            'name': 'Authenticated User Access Token',
            'domain': DOMAIN,
            'auth_id': AUTH_ID
        },
        'token': {}
    }
    token = CLIENT.create_token(setup)
    TOKEN_ID = token['id']
    print(f"Created Auth Token {token['meta']['name']} with id {token['id']}")


## Create APIs

### Bulk APIs

In [ ]:
setup = get_call_setup("bulk:accounts")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:balances")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:transactions")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:beneficiaries")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:beneficiaries")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:directdebits")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:standingorders")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:products")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:offers")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:party")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:scheduledpayments")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("bulk:statements")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

### Account APIs

In [ ]:
setup = get_call_setup("account:account")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:balance")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:transactions")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:beneficiaries")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:directdebits")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:standingorders")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:product")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:offers")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:party")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:scheduledpayments")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:statements")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:statement")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:statement_file")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
setup = get_call_setup("account:statement_transactions")
data = CLIENT.create_call(setup)
print(f"Created API {data['meta']['name']} with id {data['id']}")